# **DISTIL Usage Example: CIFAR10**

Here, we show how to use DISTIL to perform active learning on image classification tasks (CIFAR10). This notebook can be easily executed on Google Colab.

## Installation and Imports

In [ ]:
# Get DISTIL
!git clone https://github.com/decile-team/distil.git
!pip install -r distil/requirements/requirements.txt

import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import torch
import torch.optim as optim
import torch.nn.functional as F

from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset, Subset, ConcatDataset, DataLoader
from torchvision import transforms
from torchvision.datasets import cifar

sys.path.append('distil/')
from distil.active_learning_strategies import GLISTER, BADGE, EntropySampling, RandomSampling   # All active learning strategies showcased in this example
from distil.utils.models.resnet import ResNet18                                                 # The model used in our image classification example
from distil.utils.train_helper import data_train                                                # A utility training class provided by DISTIL
from distil.utils.utils import LabeledToUnlabeledDataset                                        # A utility wrapper class that removes labels from labeled PyTorch dataset objects

## Preparing CIFAR10

The CIFAR10 dataset contains 60,000 32x32 color images in 10 different classes.The 10 different classes represent airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. There are 6,000 images of each class. The training set contains 50,000 images, and the test set contains 10,000 images. Here, we do a simple setup of the CIFAR10 dataset that we will use in this example. More importantly, we define a split on CIFAR10's training set into an initial labeled seed set and an unlabeled set.

In [ ]:
data_set_name = 'CIFAR10'
download_path = '.'

# Define transforms on the dataset splits of CIFAR10. Here, we use random crops and horizontal flips for training augmentations.
# Both the train and test sets are converted to PyTorch tensors and are normalized around the mean/std of CIFAR-10.
cifar_training_transform = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
cifar_test_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

# Get the dataset objects from PyTorch. Here, CIFAR10 is downloaded, and the transform is applied when points 
# are retrieved.
cifar10_full_train = cifar.CIFAR10(download_path, train=True, download=True, transform=cifar_training_transform)
cifar10_test = cifar.CIFAR10(download_path, train=False, download=True, transform=cifar_test_transform)

# Get the dimension of the images. Here, we simply take the very first image of CIFAR10 
# and query its dimension.
dim = np.shape(cifar10_full_train[0][0])

# We now define a train-unlabeled split for the sake of the experiment. Here, we simply take 1000 points as the initial seed set.
# The rest of the points are taken as the unlabeled set. While the unlabeled set constructed here technically has labels, they 
# are only used when querying for labels. Hence, they only exist here for the sake of experimental design.
train_size = 1000
cifar10_train = Subset(cifar10_full_train, list(range(train_size)))
cifar10_unlabeled = Subset(cifar10_full_train, list(range(train_size, len(cifar10_full_train))))

# Define the number of active learning rounds to conduct, the budget, and the number of classes in CIFAR10
nclasses = 10
n_rounds = 9
budget = 500 

## Preparing the Model

Here, we use DISTIL's provided implementation of the [ResNet-18](https://arxiv.org/abs/1512.03385) architecture. We also create a model directory to store trained models in this example.

In [ ]:
net = ResNet18()
base_dir = "models"
os.makedirs(base_dir, exist_ok = True)
model_directory = os.path.join(base_dir, 'base_model.pth')

## Training an Initial Model
Here, we train an initial model. We do so by creating a training loop object on the initial seed set, the model architecture, and a list of provided arguments. We then save the initial model. Note: If you've already run the first cell, then you can simply run the second cell.

In [ ]:
# Specify additional training parameters. Here, we set the maximum number of epochs of training to 300, 
# the learning rate to 0.01, the batch size to 20, the maximum train accuracy of training to 0.99, and 
# the optimizer to stochastic gradient descent.
args = {'n_epoch':300, 'lr':float(0.01), 'batch_size':20, 'max_accuracy':0.99, 'optimizer':'sgd'} 
dt = data_train(cifar10_train, net, args)
clf = dt.train()
torch.save(clf.state_dict(), model_directory)

In [ ]:
base_dir = "models"
model_directory = os.path.join(base_dir, 'base_model.pth')
net.load_state_dict(torch.load(model_directory))
clf = net

## Active Learning Strategies

Here, we show examples of a couple active learning strategies being used in the setting of image classification.

### Random Sampling
This strategy is often used as a baseline, where we pick a subset of unlabeled points randomly. Here we create a instance of distil.active_learning_strategies.random_sampling.RandomSampling by passing following parameters:

**training_dataset** – The labeled dataset

**unlabeled_dataset** – The unlabeled dataset, which has a wrapper around it that strips the label

**net (class object)** – Model architecture used for training. Could be instance of models defined in distil.utils.models or something similar.

**nclasses (int)** – No. of classes in tha dataset

**args (dictionary)**– This dictionary should have ‘batch_size’ as a key. 'batch_size' should be such that one can exploit the benefits of tensorization while honouring the resourse constraits. This ‘batch_size’ therefore can be different than the one used for training.


In [ ]:
# Initialize the random sampling AL strategy. Note: The labels are shaved off the unlabeled dataset above to match the setting.
strategy_args = {'batch_size' : 20}
strategy = RandomSampling(cifar10_train, LabeledToUnlabeledDataset(cifar10_unlabeled), net, nclasses, strategy_args)

# Use the same training parameters as before
args = {'n_epoch':300, 'lr':float(0.01), 'batch_size':20, 'max_accuracy':0.99, 'optimizer':'sgd'} 
dt = data_train(cifar10_train, clf, args)

# Update the model used in the AL strategy with the loaded initial model
strategy.update_model(clf)

# Get the test accuracy of the initial model
acc = np.zeros(n_rounds)
acc[0] = dt.get_acc_on_set(cifar10_test)
print('Initial Testing accuracy:', round(acc[0]*100, 2), flush=True)

# User Controlled Loop
for rd in range(1, n_rounds):
    print('-------------------------------------------------')
    print('Round', rd) 
    print('-------------------------------------------------')

    # Use select() to obtain the indices in the unlabeled set that should be labeled
    cifar10_full_train.transform = cifar_test_transform       # Disable augmentation while selecting new points as to not interfere with the strategies
    idx = strategy.select(budget)
    cifar10_full_train.transform = cifar_training_transform   # Enable augmentation

    # Add the selected points to the train set. The unlabeled set shown in the next couple lines 
    # already has the associated labels, so no human labeling is needed. Again, this is because 
    # we already have the labels a priori. In real scenarios, a human oracle would need to provide 
    # then before proceeding.
    cifar10_train = ConcatDataset([cifar10_train, Subset(cifar10_unlabeled, idx)])
    remaining_unlabeled_idx = list(set(range(len(cifar10_unlabeled))) - set(idx))
    cifar10_unlabeled = Subset(cifar10_unlabeled, remaining_unlabeled_idx)

    print('Number of training points -', len(cifar10_train))

    # Update the data used in the AL strategy and the training class
    strategy.update_data(cifar10_train, LabeledToUnlabeledDataset(cifar10_unlabeled))
    dt.update_data(cifar10_train)

    # Retrain the model and update the strategy with the result
    clf = dt.train()
    strategy.update_model(clf)

    # Get new test accuracy
    acc[rd] = dt.get_acc_on_set(cifar10_test)
    print('Testing accuracy:', round(acc[rd]*100, 2), flush=True)

print('Training Completed')

# Lastly, we save the accuracies in case a comparison is warranted.
with open(os.path.join(base_dir,'random.txt'), 'w') as f:
    for item in acc:
        f.write("%s\n" % item)

### Entropy Sampling
A very basic strategy to select unlabeled points is entropy sampling, where we select samples about which the model is most uncertain by measuring the entropy of the class prediction. Hence, a valid strategy is to select those points in the unlabeled set with highest entropy (maximum uncertainty). Specifically, let $z_i$ be output from the model. By applying a softmax, we obtain probabilities that we can use: $$\sigma(z)_i = \frac{e^{z_i}}{\sum_j e^{z_j}}$$ Then, the entropy can be calculated as $$ENTROPY = -\sum_j \sigma(z)_j*log(\sigma(z)_j)$$

Here we create a instance of distil.active_learning_strategies.entropy_sampling.EntropySampling with the same parameters passed to distil.active_learning_strategies.random_sampling.RandomSampling.

**Reloading Base Model & Data**

We make sure the fixture is the same by repeating the same setup.

In [ ]:
data_set_name = 'CIFAR10'
download_path = '.'

cifar_training_transform = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
cifar_test_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

cifar10_full_train = cifar.CIFAR10(download_path, train=True, download=True, transform=cifar_training_transform)
cifar10_test = cifar.CIFAR10(download_path, train=False, download=True, transform=cifar_test_transform)

dim = np.shape(cifar10_full_train[0][0])

train_size = 1000
cifar10_train = Subset(cifar10_full_train, list(range(train_size)))
cifar10_unlabeled = Subset(cifar10_full_train, list(range(train_size, len(cifar10_full_train))))

nclasses = 10
n_rounds = 9    
budget = 500 

net = ResNet18()

In [ ]:
base_dir = "models"
model_directory = os.path.join(base_dir, 'base_model.pth')
net.load_state_dict(torch.load(model_directory))
clf = net

In [ ]:
# Initialize the random sampling AL strategy. Note: The labels are shaved off the unlabeled dataset above to match the setting.
strategy_args = {'batch_size' : 20}
strategy = EntropySampling(cifar10_train, LabeledToUnlabeledDataset(cifar10_unlabeled), net, nclasses, strategy_args)

# Use the same training parameters as before
args = {'n_epoch':300, 'lr':float(0.01), 'batch_size':20, 'max_accuracy':0.99, 'optimizer':'sgd'} 
dt = data_train(cifar10_train, clf, args)

# Update the model used in the AL strategy with the loaded initial model
strategy.update_model(clf)

# Get the test accuracy of the initial model
acc = np.zeros(n_rounds)
acc[0] = dt.get_acc_on_set(cifar10_test)
print('Initial Testing accuracy:', round(acc[0]*100, 2), flush=True)

# User Controlled Loop
for rd in range(1, n_rounds):
    print('-------------------------------------------------')
    print('Round', rd) 
    print('-------------------------------------------------')

    # Use select() to obtain the indices in the unlabeled set that should be labeled
    cifar10_full_train.transform = cifar_test_transform       # Disable augmentation while selecting new points as to not interfere with the strategies
    idx = strategy.select(budget)
    cifar10_full_train.transform = cifar_training_transform   # Enable augmentation

    # Add the selected points to the train set. The unlabeled set shown in the next couple lines 
    # already has the associated labels, so no human labeling is needed. Again, this is because 
    # we already have the labels a priori. In real scenarios, a human oracle would need to provide 
    # then before proceeding.
    cifar10_train = ConcatDataset([cifar10_train, Subset(cifar10_unlabeled, idx)])
    remaining_unlabeled_idx = list(set(range(len(cifar10_unlabeled))) - set(idx))
    cifar10_unlabeled = Subset(cifar10_unlabeled, remaining_unlabeled_idx)

    print('Number of training points -', len(cifar10_train))

    # Update the data used in the AL strategy and the training class
    strategy.update_data(cifar10_train, LabeledToUnlabeledDataset(cifar10_unlabeled))
    dt.update_data(cifar10_train)

    # Retrain the model and update the strategy with the result
    clf = dt.train()
    strategy.update_model(clf)

    # Get new test accuracy
    acc[rd] = dt.get_acc_on_set(cifar10_test)
    print('Testing accuracy:', round(acc[rd]*100, 2), flush=True)

print('Training Completed')

#Saving accuracies for further analysis
with open(os.path.join(base_dir,'entropy.txt'), 'w') as f:
    for item in acc:
        f.write("%s\n" % item)

### BADGE
This method is based on the paper [Deep Batch Active Learning by Diverse, Uncertain Gradient Lower Bounds](https://arxiv.org/abs/1906.03671). The strategy is meant to select points that are both diverse (e.g., their embeddings span multiple directions) and uncertain (e.g., their contribution to the loss is large). The following steps are taken:

* Calculate the pseudo-label for each point in the unlabeled set. The pseudo-label is the class with the highest probability.
* Compute the cross-entropy loss for each point in the unlabeled set using this pseudo-label.
* Obtain the resulting loss gradients on the last linear layer of the model for each point. (These are referred to as the hypothesized loss gradients.)
* Using these gradients as a form of embedding for each unlabeled point, run k-means++ initialization on this embedding set, retrieving $k$ centers. Each center is a point from the unlabeled set, and $k$ represents the active learning budget.
* Request labels for the $k$ points whose embeddings were selected.

Here we create a instance of distil.active_learning_strategies.badge.BADGE with same parameters passed to distil.active_learning_strategies.random_sampling.RandomSampling.

**Reloading Base Model & Data**

In [ ]:
data_set_name = 'CIFAR10'
download_path = '.'

cifar_training_transform = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
cifar_test_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

cifar10_full_train = cifar.CIFAR10(download_path, train=True, download=True, transform=cifar_training_transform)
cifar10_test = cifar.CIFAR10(download_path, train=False, download=True, transform=cifar_test_transform)

dim = np.shape(cifar10_full_train[0][0])

train_size = 1000
cifar10_train = Subset(cifar10_full_train, list(range(train_size)))
cifar10_unlabeled = Subset(cifar10_full_train, list(range(train_size, len(cifar10_full_train))))

nclasses = 10
n_rounds = 9    ##Number of rounds to run active learning
budget = 500 

net = ResNet18()

In [ ]:
base_dir = "models"
model_directory = os.path.join(base_dir, 'base_model.pth')
net.load_state_dict(torch.load(model_directory))
clf = net

In [ ]:
# Initialize the random sampling AL strategy. Note: The labels are shaved off the unlabeled dataset above to match the setting.
strategy_args = {'batch_size' : 20}
strategy = BADGE(cifar10_train, LabeledToUnlabeledDataset(cifar10_unlabeled), net, nclasses, strategy_args)

# Use the same training parameters as before
args = {'n_epoch':300, 'lr':float(0.01), 'batch_size':20, 'max_accuracy':0.99, 'optimizer':'sgd'} 
dt = data_train(cifar10_train, clf, args)

# Update the model used in the AL strategy with the loaded initial model
strategy.update_model(clf)

# Get the test accuracy of the initial model
acc = np.zeros(n_rounds)
acc[0] = dt.get_acc_on_set(cifar10_test)
print('Initial Testing accuracy:', round(acc[0]*100, 2), flush=True)

# User Controlled Loop
for rd in range(1, n_rounds):
    print('-------------------------------------------------')
    print('Round', rd) 
    print('-------------------------------------------------')

    # Use select() to obtain the indices in the unlabeled set that should be labeled
    cifar10_full_train.transform = cifar_test_transform       # Disable augmentation while selecting new points as to not interfere with the strategies
    idx = strategy.select(budget)
    cifar10_full_train.transform = cifar_training_transform   # Enable augmentation

    # Add the selected points to the train set. The unlabeled set shown in the next couple lines 
    # already has the associated labels, so no human labeling is needed. Again, this is because 
    # we already have the labels a priori. In real scenarios, a human oracle would need to provide 
    # then before proceeding.
    cifar10_train = ConcatDataset([cifar10_train, Subset(cifar10_unlabeled, idx)])
    remaining_unlabeled_idx = list(set(range(len(cifar10_unlabeled))) - set(idx))
    cifar10_unlabeled = Subset(cifar10_unlabeled, remaining_unlabeled_idx)

    print('Number of training points -', len(cifar10_train))

    # Update the data used in the AL strategy and the training class
    strategy.update_data(cifar10_train, LabeledToUnlabeledDataset(cifar10_unlabeled))
    dt.update_data(cifar10_train)

    # Retrain the model and update the strategy with the result
    clf = dt.train()
    strategy.update_model(clf)

    # Get new test accuracy
    acc[rd] = dt.get_acc_on_set(cifar10_test)
    print('Testing accuracy:', round(acc[rd]*100, 2), flush=True)

print('Training Completed')

#Saving accuracies for further analysis
with open(os.path.join(base_dir,'badge.txt'), 'w') as f:
    for item in acc:
        f.write("%s\n" % item)

### GLISTER
This is implemetation of GLISTER-ACTIVE from the paper [GLISTER: Generalization based Data Subset Selection for Efficient and Robust Learning](https://arxiv.org/abs/2012.10630). GLISTER tries to solve the following bi-level optimisation problem:
\begin{equation*}
\overbrace{\underset{{S \subseteq {\mathcal U}, |S| \leq k}}{\operatorname{argmax\hspace{0.7mm}}} LL_V(\underbrace{\underset{\theta}{\operatorname{argmax\hspace{0.7mm}}} LL_T( \theta, S)}_{inner-level}, {\mathcal V})}^{outer-level}
\end{equation*}
where is $S$ is set of points selected at each round, ${\mathcal V}$ could be a dedicated validation set with labled points or could be union of labeled and unlabeled points with hypothesized labels, and $k$ is the budget.

Solving this problem directly is almost intractable due to the bi-level nature (e.g., repeatedly having to solve the inner problem); therefore, GLISTER resorts to one-step approxiations. To start, we set $S^0$ as empty set and incrementally build it as $S^k = S^{k-1} \cup e$, where $e$ is $\underset{e}{\operatorname{argmax\hspace{0.7mm}}} G_{\theta}(e | S^k)$, defined below: $$G_{\theta}(e | S^k) = LL_{V}(\theta^{k}, {\mathcal V})$$

Concurrently, GLISTER updates the model parameters via the aforementioned one-step approximation:

 $$\theta^k \leftarrow \theta^{k-1} -  \eta \nabla_{\theta} LL_T(\hat{\theta}, e)$$ where $\hat{\theta}$ are the parameters of the model at the begining of the selection.

To prevent overfitting, we can add regularization to GLISTER, which can be set by **_typeOf_**. **_typeOf_** can be set to **None** for normal GLISTER (default), **'Rand'** for replacing **_lam_** fraction of points with random points, **'Diversity'** for adding a diversity set function while computing gain, and **'FacLoc'** for adding a facility location set function while computing gain. **_lam_** for both **'Diversity'** and **'FacLoc'** determines the weightage given to them while computing the gain.

Here, we create a instance of distil.active_learning_strategies.glister.GLISTER with the same parameters passed to distil.active_learning_strategies.random_sampling.RandomSampling. Additionally, we add to the **args** dictionary the required keys ‘batch_size’ and ‘lr’. ‘lr’ should be the learning rate used for training. Lastly, the following parameters can be passed as previously described:

**validation_dataset (torch.utils.data.Dataset, optional)** – An optional validation dataset.

**typeOf (str, optional)** – Determines the type of regularizer to be used. The default is **None**. For a random regularizer, use **‘Rand’**. To use the facility location set function as a regularizer, use **‘FacLoc’**. To use a diversity set function as a regularizer, use **‘Diversity’**.

**lam (float, optional)** – Determines the amount of regularization to be applied. Mandatory if is not **_typeOf_=None** and, by default, is set to **None**. For random regularizers, use values between 0 and 1 as it determines fraction of points replaced by random points. For both **‘Diversity’** and **‘FacLoc’**, **_lam_** determines the weightage given to them while computing the gain.

**kernel_batch_size (int, optional)** – For the **'Diversity'** and **'FacLoc'** regularizer versions, a similarity kernel is computed, which entails creating a 3-D Torch tensor of dimensions $kernel\_batch\_size^{2}\times(feature\ dimension)$. Again, **_kernel_batch_size_** should be such that one can exploit the benefits of tensorization while honouring the resourse constraints.

**Reloading Base Model & Data**

In [ ]:
data_set_name = 'CIFAR10'
download_path = '.'

cifar_training_transform = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
cifar_test_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
cifar_target_transform = transforms.ToTensor()

cifar10_full_train = cifar.CIFAR10(download_path, train=True, download=True, transform=cifar_training_transform, target_transform=torch.tensor)
cifar10_test = cifar.CIFAR10(download_path, train=False, download=True, transform=cifar_test_transform, target_transform=torch.tensor)

dim = np.shape(cifar10_full_train[0][0])

train_size = 1000
cifar10_train = Subset(cifar10_full_train, list(range(train_size)))
cifar10_unlabeled = Subset(cifar10_full_train, list(range(train_size, len(cifar10_full_train))))

nclasses = 10
n_rounds = 9    ##Number of rounds to run active learning
budget = 500 

net = ResNet18()

In [ ]:
base_dir = "models"
model_directory = os.path.join(base_dir, 'base_model.pth')
net.load_state_dict(torch.load(model_directory))
clf = net

In [ ]:
#Initializing Strategy Class
strategy_args = {'batch_size' : 20, 'lr' : 0.01}
strategy = GLISTER(cifar10_train, LabeledToUnlabeledDataset(cifar10_unlabeled), net, nclasses, strategy_args)

# Use the same training parameters as before
args = {'n_epoch':300, 'lr':float(0.01), 'batch_size':20, 'max_accuracy':0.99, 'optimizer':'sgd'} 
dt = data_train(cifar10_train, clf, args)

# Update the model used in the AL strategy with the loaded initial model
strategy.update_model(clf)

# Get the test accuracy of the initial model
acc = np.zeros(n_rounds)
acc[0] = dt.get_acc_on_set(cifar10_test)
print('Initial Testing accuracy:', round(acc[0]*100, 2), flush=True)

# User Controlled Loop
for rd in range(1, n_rounds):
    print('-------------------------------------------------')
    print('Round', rd) 
    print('-------------------------------------------------')

    # Use select() to obtain the indices in the unlabeled set that should be labeled
    cifar10_full_train.transform = cifar_test_transform       # Disable augmentation while selecting new points as to not interfere with the strategies
    idx = strategy.select(budget)
    cifar10_full_train.transform = cifar_training_transform   # Enable augmentation

    # Add the selected points to the train set. The unlabeled set shown in the next couple lines 
    # already has the associated labels, so no human labeling is needed. Again, this is because 
    # we already have the labels a priori. In real scenarios, a human oracle would need to provide 
    # then before proceeding.
    cifar10_train = ConcatDataset([cifar10_train, Subset(cifar10_unlabeled, idx)])
    remaining_unlabeled_idx = list(set(range(len(cifar10_unlabeled))) - set(idx))
    cifar10_unlabeled = Subset(cifar10_unlabeled, remaining_unlabeled_idx)

    print('Number of training points -', len(cifar10_train))

    # Update the data used in the AL strategy and the training class
    strategy.update_data(cifar10_train, LabeledToUnlabeledDataset(cifar10_unlabeled))
    dt.update_data(cifar10_train)

    # Retrain the model and update the strategy with the result
    clf = dt.train()
    strategy.update_model(clf)

    # Get new test accuracy
    acc[rd] = dt.get_acc_on_set(cifar10_test)
    print('Testing accuracy:', round(acc[rd]*100, 2), flush=True)

print('Training Completed')

#Saving accuracies for further analysis
with open(os.path.join(base_dir,'glister.txt'), 'w') as f:
    for item in acc:
        f.write("%s\n" % item)

## Visualizing the Results

If all strategies have run to completion, you can run the following cell to view the results.

In [ ]:
# Load the accuracies previously obtained
with open(os.path.join(base_dir,'entropy.txt'), 'r') as f:
  acc_ = f.readlines()
acc_en = [round(float(x)*100, 2) for x in acc_]
with open(os.path.join(base_dir,'badge.txt'), 'r') as f:
  acc_ = f.readlines()
acc_bd = [round(float(x)*100, 2) for x in acc_]
with open(os.path.join(base_dir,'glister.txt'), 'r') as f:
  acc_ = f.readlines()
acc_gl = [round(float(x)*100, 2) for x in acc_]
with open(os.path.join(base_dir,'random.txt'), 'r') as f:
  acc_ = f.readlines()
acc_rd = [round(float(x)*100, 2) for x in acc_]

# Plot them using matplotlib
x_axis = np.array([train_size+budget*i for i in range(n_rounds)])
plt.figure()
plt.plot(x_axis, acc_gl, 'b-', label='GLISTER RAND',marker='o')
plt.plot(x_axis, acc_en, 'g-', label='UNCERTAINITY',marker='o')
plt.plot(x_axis, acc_bd, 'c', label='BADGE',marker='o')
plt.plot(x_axis, acc_rd, 'r', label='RANDOM',marker='o')

plt.legend()
plt.xlabel('No of Images')
plt.ylabel('Test Accuracy')
plt.title('DISTIL_CIFAR10')